In [ ]:
import sys
!{sys.executable} -m pip install matplotlib

In [2]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

2021-12-21 23:34:30.150102: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2021-12-21 23:35:30.552543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 23:35:30.643722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 23:35:30.644478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 23:35:30.646578: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


2021-12-21 23:35:32.915704: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-21 23:35:32.960138: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
x_train = []
x_test = []
y_train = []
y_test = []

randomized_train_dataset = dataset["train"]

iterator = iter(randomized_train_dataset)
while(1):
  val = iterator.get_next_as_optional()  
  if(val.has_value().numpy() == False):
    break
  data, lbl = val.get_value()
  x_train.append(str(data.numpy().decode("utf-8")))
  y_train.append(lbl.numpy())


randomized_test_dataset = dataset["test"]

iterator = iter(randomized_test_dataset)

while(1):
  val = iterator.get_next_as_optional()  
  if(val.has_value().numpy() == False):
    break
  data, lbl = val.get_value()
  x_test.append(str(data.numpy().decode("utf-8")))
  y_test.append(lbl.numpy())

In [7]:
VOCAB_SIZE = 5000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [8]:
v = encoder(x_train)
vocab = np.array(encoder.get_vocabulary())

In [9]:
indices = tf.range(start=0, limit=v.shape[1], dtype=tf.int32)

In [10]:
shuffled_x_train=[]
for i in range(0, v.shape[0]):
    shuffled_indices = tf.random.shuffle(indices)
    shuffled_x = tf.gather(v[i], shuffled_indices)
#     print(shuffled_x)
#     print(" ".join(vocab[shuffled_x]))
    shuffled_x_train.append(" ".join(vocab[shuffled_x]))

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((shuffled_x_train, y_train))

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [13]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [14]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [16]:
history = model.fit(train_dataset, epochs=5,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/5


2021-12-21 23:37:36.354739: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


391/391 [==============================] - 77s 153ms/step - loss: 0.6467 - accuracy: 0.5588 - val_loss: 0.4234 - val_accuracy: 0.8089
Epoch 2/5
391/391 [==============================] - 55s 142ms/step - loss: 0.3550 - accuracy: 0.8446 - val_loss: 0.3302 - val_accuracy: 0.8547
Epoch 3/5
391/391 [==============================] - 55s 142ms/step - loss: 0.2725 - accuracy: 0.8898 - val_loss: 0.3308 - val_accuracy: 0.8438
Epoch 4/5
391/391 [==============================] - 55s 142ms/step - loss: 0.2369 - accuracy: 0.9064 - val_loss: 0.3248 - val_accuracy: 0.8375
Epoch 5/5
391/391 [==============================] - 55s 141ms/step - loss: 0.2132 - accuracy: 0.9191 - val_loss: 0.3186 - val_accuracy: 0.8635
